In [1]:
import selenium
import re
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from cmath import nan
from geopy.geocoders import Nominatim

def initialize():
    #CHROMEDRIVER_PATH = '/opt/WebDriver/bin'
    WINDOW_SIZE = "1000,1024"


    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    #chrome_options.binary_location = CHROME_PATH

    # Creating a chromedriver instance
    driver = webdriver.Chrome('D:\DATA\DV\chromedriver.exe')#executable_path=CHROMEDRIVER_PATH, 

    driver.get("https://www.boliga.dk/resultat?zipCodes=2770")
    driver.implicitly_wait(2)#It instructs the webdriver to wait for a certain amount of time before elements load on the webpage.

    driver.find_element(By.XPATH, "//button[contains(text(), 'Tillad alle')]").click()

    return driver

driver = initialize()
    
def zip_cph():
    zip_df = pd.read_csv('zip_cph.csv', header=None)

    postal_areas = []

    for i in zip_df[1]:
        numbers = i.replace("'", "")
        numbers = numbers.replace(",", "")
        numbers = numbers.split()
        for number in numbers:
            number = number.replace("[", "")
            number = number.replace("]", "")
            postal_areas.append(number)
    return postal_areas

def zip_cph_extended():
    zip_df = pd.read_csv('zip_cph_extended.csv', header=None)

    postal_areas = []

    for i in zip_df[1]:
        numbers = i.replace("'", "")
        numbers = numbers.replace(",", "")
        numbers = numbers.split()
        for number in numbers:
            number = number.replace("[", "")
            number = number.replace("]", "")
            postal_areas.append(number)
    return postal_areas


def searchzip(postal_areas):
    searchzip = []
    for area in postal_areas:
        a = ("https://www.boliga.dk/resultat?zipCodes=" + area)
        print(a)
        searchzip.append(a)
    return searchzip

def get_houselinks(searchzip):
    proceed = True
    houselinks = []

    for search in searchzip:
        time.sleep(1)
        driver.get(search)
        proceed = True

        while proceed:
            time.sleep(1)
            houses = driver.find_elements(By.XPATH, "//a[@class = 'house-list-item']")
            for house in houses:
                houselink = house.get_attribute("href")
                houselinks.append(houselink)
            try:
                nextbutton = driver.find_element(By.XPATH, "//a[not(contains(@class, 'disabled')) and contains(text(), 'Næste')]")
                nextbutton.click()
            except:
                proceed = False
    return houselinks

def create_dataframe():
    df = pd.DataFrame(columns = ['Type', 'Address', 'Price', 'Sqm Price', 'Size', 'Living Area', 'Ground Area', 'Rooms', 'Ownership Expense', 'Year Built', 'Story', 'Energy Label', 'Description'])
    return df

def scrape(houselinks, df):
    for link in houselinks:
        try:
            driver.get(link)
            driver.implicitly_wait(2)

            try:
                house_address = driver.find_element(By.XPATH, "//*[@class = 'house-place ng-star-inserted']").text
            except: 
                house_address = "nan"

            try:
                type = driver.find_element(By.XPATH, "//span[@class = 'house-name']").text
            except: 
                type = "nan"

            try:
                house_price = driver.find_element(By.XPATH, "//*[@class = 'text-right house-price d-flex align-items-center justify-content-end ng-star-inserted']").text
            except: 
                house_price = "nan" 

            try:   
                price_sqm = driver.find_element(By.XPATH, "//*[@class = 'd-block text-right text-muted house-sqm-price ng-star-inserted']").text
            except: 
                price_sqm = "nan"

            try:
                house_size = driver.find_element(By.XPATH, "//span[contains(text(), 'Boligstørrelse:')]").text
            except: 
                house_size = "nan"

            try:
                living_area = driver.find_element(By.XPATH, "//span[contains(text(), 'Boligareal, tinglyst:')]").text
            except: 
                living_area = "nan"

            try:
                ground_area = driver.find_element(By.XPATH, "//span[contains(text(), 'Grundstørrelse:')]").text
            except: 
                ground_area = "nan"

            try:
                rooms = driver.find_element(By.XPATH, "//span[contains(text(), 'Værelser: ')]").text
            except: 
                rooms = "nan"

            try:
                ownership_expense = driver.find_element(By.XPATH, "//span[contains(text(), 'Ejerudgift:')]").text
            except: 
                ownership_expense = "nan"

            try:
                see_more_button = driver.find_element(By.XPATH, "//div[contains(text(), 'Vis mere')]")
                driver.execute_script('arguments[0].click()', see_more_button)
            except:
                pass

            try:
                year_built = driver.find_element(By.XPATH, "//span[contains(text(), 'Byggeår:')]").text
            except: 
                year_built = "nan"

            try:
                story = driver.find_element(By.XPATH, "//span[contains(text(), 'Sal:')]").text
            except:
                story = "st"

            try:
                energy_label = driver.find_element(By.XPATH, "//span[contains(text(), 'Energimærke:')]").text
            except:
                energy_label = "nan"  

            try:
                description = driver.find_element(By.XPATH, "//div[contains(@class, 'shared-property-information')]").text
            except:
                description = "nan"  

            df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': year_built, 'Story': story, 'Energy Label': energy_label, 'Description': description}, 
                        ignore_index = True) 
        except:
            pass
        
    return df

def get_coordinates(df):
    geolocator = Nominatim(user_agent="my_user_agent")
    addresses = df['Address'].values
    country ="Dk"
    longitude = []
    latitude=[]
    cleaned_address = []

    for address in addresses:
        address = re.sub("(?i)\d+\.+\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = re.sub("\d+\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = re.sub("\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = address.replace(",", "")
        address = address.replace(".", "")
        print(address)
        cleaned_address.append(address)
        try:
            loc = geolocator.geocode(address)
            if loc:
                print(address)
                print(loc)
                print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
                longitude.append(loc.longitude)
                latitude.append(loc.latitude)
            else:
                print(address)
                print("not found")
                longitude.append("nan")
                latitude.append("nan")
        except:
            print(address)
            print("not found")
            longitude.append("nan")
            latitude.append("nan")

    df["Longitude"] = longitude
    df["Latitude"] = latitude
    df["Cleaned_address"] = cleaned_address

def save_df(df, name):
    df.to_csv(name+'.csv') 

def append_df(df, df2):
   data = df.append(df2, ignore_index = True)
   return data  


def run_script():

    zips = searchzip(zip_cph())

    #zips = searchzip(zip_cph_extended()) uncomment if the scraper should be ran for the outskirts

    houselinks = get_houselinks(zips)

    df = create_dataframe()

    scraped_df = scrape(houselinks, df)

    df = get_coordinates(scraped_df)

    save_df(df, "scraped_data") # provide the name on how to save the .csv

    return df

run_script()


C:\Users\kovac\AppData\Local\Temp\ipykernel_1004\1747664938.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:\DATA\DV\chromedriver.exe')#executable_path=CHROMEDRIVER_PATH,


https://www.boliga.dk/resultat?zipCodes=2700
https://www.boliga.dk/resultat?zipCodes=1050
https://www.boliga.dk/resultat?zipCodes=1051
https://www.boliga.dk/resultat?zipCodes=1052
https://www.boliga.dk/resultat?zipCodes=1053
https://www.boliga.dk/resultat?zipCodes=1054
https://www.boliga.dk/resultat?zipCodes=1055
https://www.boliga.dk/resultat?zipCodes=1056
https://www.boliga.dk/resultat?zipCodes=1057
https://www.boliga.dk/resultat?zipCodes=1058
https://www.boliga.dk/resultat?zipCodes=1059
https://www.boliga.dk/resultat?zipCodes=1060
https://www.boliga.dk/resultat?zipCodes=1061
https://www.boliga.dk/resultat?zipCodes=1062
https://www.boliga.dk/resultat?zipCodes=1063
https://www.boliga.dk/resultat?zipCodes=1064
https://www.boliga.dk/resultat?zipCodes=1065
https://www.boliga.dk/resultat?zipCodes=1066
https://www.boliga.dk/resultat?zipCodes=1067
https://www.boliga.dk/resultat?zipCodes=1068
https://www.boliga.dk/resultat?zipCodes=1069
https://www.boliga.dk/resultat?zipCodes=1070
https://ww


KeyboardInterrupt

